In [1]:
from ensemble_boxes import *
import pandas as pd
import numpy as np
from tqdm import tqdm

In [57]:
#submission files 에 ensemble할 submission.csv 파일들 경로 입력
submission_files = ['./ensem_subs/cascade_rcnn_swinl_fpn_aug12_1024_epoch_14.csv',
                    './ensem_subs/faster_rcnn_swinL_PAFPN_aug12_1024.csv',
                    './ensem_subs/faster_rcnn_swinl_FPN_aug12_1024.csv']

submissions = [pd.read_csv(s) for s in submission_files]

WBF

In [37]:
WBF_IOU_THR = 0.6
# WBF_SKIP_BOX_THR = 0.0
WBF_CONF_TYPE = 'avg'

output_csv_path = './ensembled_submission.csv'

In [90]:
image_ids = submissions[0]['image_id'].tolist()
prediction_strings = []
file_names = []

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in tqdm(enumerate(image_ids)):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submissions:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / 1024.0
            box[1] = float(box[1]) / 1024.0
            box[2] = float(box[2]) / 1024.0
            box[3] = float(box[3]) / 1024.0
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

    # 예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        # Perform Ensemble
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=WBF_IOU_THR, conf_type=WBF_CONF_TYPE)
        
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * 1024) + ' ' + str(box[1] * 1024) + ' ' + str(box[2] * 1024) + ' ' + str(box[3] * 1024) + ' '
    
    prediction_strings.append(prediction_string)
    
    file_names.append(image_id)

In [92]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.head()

,PredictionString,image_id
0,7 0.9993438124656677 607.0755615234375 523.212...,test/0000.jpg
1,5 0.9797833561897278 347.2628173828125 247.238...,test/0001.jpg
2,1 0.9812363386154175 289.4942932128906 305.360...,test/0002.jpg
3,9 0.7993578314781189 166.6297607421875 265.869...,test/0003.jpg
4,1 0.767069399356842 196.54776000976562 259.472...,test/0004.jpg


In [95]:
submission.to_csv(output_csv_path, index=None)